In [6]:
data_new

,clickHistory,current_news,label,id
0,N61901 N28165 N5994 N32458 N30740 N63732 N2290...,N57352,0,U7615
0,N61901 N28165 N5994 N32458 N30740 N63732 N2290...,N37113,0,U7615
0,N61901 N28165 N5994 N32458 N30740 N63732 N2290...,N15218,0,U7615
0,N61901 N28165 N5994 N32458 N30740 N63732 N2290...,N63863,0,U7615
0,N61901 N28165 N5994 N32458 N30740 N63732 N2290...,N54656,0,U7615
...,...,...,...,...
156964,N33347 N58459 N45956 N35685 N15456 N47193 N388...,N40349,0,U63859
156964,N33347 N58459 N45956 N35685 N15456 N47193 N388...,N16697,0,U63859
156964,N33347 N58459 N45956 N35685 N15456 N47193 N388...,N53867,0,U63859
156964,N33347 N58459 N45956 N35685 N15456 N47193 N388...,N61497,0,U63859


# 1.读取行为数据

In [1]:
print('start')
import pandas as pd

import time
start = time.time()
train_behavior = pd.read_csv('./data/train/behaviors.tsv',sep ='\t',header=None)
###处理 behaviors.tsv 行为数据
train_behavior.columns = ['index','id','timeStamp','clickHistory','impressionLog']
impressionLog = train_behavior['impressionLog'].str.split(' ', expand=True).stack().reset_index(level = 1,drop = True)
key, label = impressionLog.str.split('-', 1).str
data_new = pd.DataFrame()
data_new= train_behavior[['clickHistory']].join(pd.DataFrame(key,columns=['current_news']))
data_new['label'] = label
data_new['id'] = train_behavior['id']
data_new.dropna(axis=0, how='any', inplace=True)


print(data_new.shape[0])
print('time is : ',time.time() - start)


start


/home/huangzhen/anaconda3/envs/news/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  # Remove the CWD from sys.path while we load stuff.


5723002
time is :  44.732322454452515


# 2.读取新闻数据

In [2]:
# coding=utf-8
title_len = 10
title = pd.read_csv('./data/train/news.tsv',sep='\t',header=None, encoding='utf-8')
title.columns = ['id','category','sub_category','title','abstract','url','title_entity','abstract_entity']
title = title[['id','title']]
arr = []
for index ,row in title.iterrows():
    text = row['title'].split(' ')
    if(len(text) < title_len):
        text.extend(['0'] * (title_len - len(text)))
    else:
        text = text[:title_len]
    text.append('[SEP]')
    arr.append(text)
title['title'] = arr

# 3.读取邻居数据

In [3]:
neighbor = pd.read_csv('./data/nearst_4.csv')
neighbor['nearst'] = neighbor['nearst'].apply(lambda x : eval(x))


# 4.多线程生成具体行为数据

In [ ]:
import re
import numpy as np
import pandas as pd

punctuation = '!,;:?"\''
def removeFormat(text):
    result = re.sub(r'[{}]+'.format(punctuation),'',text).strip().lower()
    return result
def sample_data(data,name):
    x_id = []
    x_news = []
    x_title = []
    x_label = []
    maxCount = 15
    step = 0
    for index,row in (data.iterrows()):
        if(step > -1 and step % 1000 == 0):
            print(name,':',step)
        text = []
        
        if(type( row['clickHistory']) == float): continue
        history = row['clickHistory'].split(' ') 
        idxs = list(range(len(history)))
        np.random.shuffle(idxs)
        count = 0
        for x in idxs:
            if(count == maxCount):break
            tit = title[title['id']==history[x]]['title'].values[0]
            
            if type(tit) == float:continue
            text.extend(tit)
            
            count += 1
        
        tit = title[title['id'] == row['current_news']]['title'].values[0]
        
            
        if type(tit) == float:continue
        x_news.append(text)
        x_title.append(tit)
        x_label.append(int(row['label']))
        x_id.append(row['id'])
        step += 1
       
    all_data = pd.DataFrame()
    
    all_data['id'] = x_id
    all_data['history'] = x_news
    all_data['current'] = x_title
    all_data['label'] = x_label

    all_data['history'] = all_data['history'].apply(lambda x : [removeFormat(y) for y in x])
    all_data['current'] = all_data['current'].apply(lambda x : [removeFormat(y) for y in x])
    all_data.to_csv('./data/train/seprate_10/train_{0}.csv'.format(name),index=False)
    print('curernt over: ',name)
import threading
import multiprocessing
from multiprocessing import Process
pool_thread = []
groups = 46
pool = multiprocessing.Pool(processes=groups)
data_temp = data_new.iloc[:]
for index in range(groups):
    count_data = data_temp.shape[0] // groups
    pool.apply_async(sample_data,(data_temp.iloc[index * count_data:(index+1)*count_data,:],index+1))

pool.close()
pool.join() 

print('all over!')


1 : 0
2 : 0
3 : 0
4 : 0
5 : 0
6 : 0
7 : 0
8 : 0
9 : 0
10 : 0
11 : 0
12 : 0
13 : 0
14 : 0
15 : 0
16 : 0
17 : 0
18 : 0
19 : 0
20 : 0
21 : 0
22 : 0
23 : 0
24 : 0
25 : 0
26 : 0
27 : 0
28 : 0
29 : 0
30 : 0
31 : 0
32 : 0
33 : 0
34 : 0
35 : 0
36 : 0
37 : 0
38 : 0
39 : 0
40 : 0
41 : 0
42 : 0
43 : 0
44 : 0
45 : 0
46 : 0
7 : 1000
28 : 1000
36 : 1000
8 : 1000
30 : 1000
6 : 1000
21 : 1000
25 : 1000
32 : 1000
18 : 1000
27 : 1000
37 : 1000
24 : 1000
13 : 1000
23 : 1000
31 : 1000
15 : 1000
26 : 1000
34 : 1000
17 : 1000
14 : 1000
39 : 1000
46 : 1000
4 : 1000
41 : 1000
43 : 1000
42 : 1000
40 : 1000
16 : 1000
22 : 1000
45 : 1000
38 : 1000
2 : 1000
33 : 1000
29 : 1000
3 : 1000
10 : 1000
11 : 1000
5 : 100019 
: 1000
35 : 1000
20 : 1000
12 : 1000
9 : 1000
1 : 1000
44 : 1000
13 : 2000
8 : 2000
41 : 2000
21 : 2000
7 : 2000
34 : 2000
28 : 2000
45 : 2000
27 : 2000
32 : 2000
36 : 2000
39 : 200030
 : 2000
42 : 2000
46 : 2000
25 : 2000
24 : 2000
15 : 2000
6 : 2000
31 : 2000
10 : 2000
23 : 2000
33 : 2000
22 : 2000

36 : 18000
11 : 17000
16 : 17000
45 : 18000
20 : 18000
43 : 18000
21 : 18000
18 : 17000
39 : 18000
15 : 17000
29 : 18000
24 : 18000
2 : 15000
44 : 18000
3 : 15000
33 : 18000
22 : 18000
4 : 16000


# 5.整合所有多线程数据 

In [ ]:
import numpy as np
import pandas as pd
data = pd.DataFrame()
for index in range(1,47):
    print(index)
    now = pd.read_csv('./data/test/seprate_10/test_{0}.csv'.format(index))
    data = pd.concat([data,now],axis=0)
data.to_csv('./data/test/seprate_10/all_10.csv',index=False)

# 6.训练数据采样

In [ ]:
k = 1 ##邻居数量
def sample_neg(data,number):
    k = 1
    shape = data.shape[0]
    index = 0
    index_arr = []
    index_neighbor = []
    while index < shape:
        flag = True
        for his in range(k):
            if data.iloc[index+his]['label'] != 0:
                flag = False
                break
        if(flag):
            for pos in range(k):
                index_neighbor.append(neighbor[neighbor['id'] == data.iloc[pos+index]['id']]['nearst'][:k])
                index_arr.append(pos + index)
            index += k
            if(index + k >= shape):break
            while(data.iloc[index]['label'] != 1):
                index += 1
                if(index + k >= shape):break
            if(index + k >= shape):break
            index_arr.append(index)
            index_neighbor.append(neighbor[neighbor['id'] == data.iloc[index]['id']]['nearst'][:k])
        else:
            index += 1
        if(len(index_arr) >= number):break
        if(len(index_arr) % 2000 == 0):print(len(index_arr))
        if(index + k >= shape):break 
    return index_arr,index_neighbor
def remove_sep(arr):
    temp = []
    for s in arr:
        if s != '[sep]':
            temp.append(s)
    temp = temp[:165]
    return temp
def upper_sep(arr):
    temp = []
    for s in arr:
        if s == '[sep]':
            temp.append('[SEP]')
        else:
            temp.append(s)
    temp = temp[:165]
    return temp
data = pd.read_csv('./data/train/seprate_10/all_10.csv')
index_arr,index_neighbbor = sample_neg(data,number=100000)
co_neigh = []
for index in range(len(index_neighbor)):
    co_neigh.append(data[data['id'] == index_neighbor[index]]['history'])
data = data.iloc[index_arr]

data['history'] = data['history'].apply(lambda y :' '.join(upper_sep(eval(y))))
data['current'] = data['current'].apply(lambda y :' '.join(remove_sep(eval(y))))
data['neighbor'] = co_neigh
data['neighbor'] = data['neighbor'].apply(lambda y :' '.join(upper_sep(eval(y))))
np.save('./data/bert/15_10/train_8w.npy',data.values.tolist()[:50000])
np.save('./data/bert/15_10/dev_2w.npy',data.values.tolist()[50000:])


data = pd.read_csv('./data/test/seprate_10/all_10.csv')
co_neigh = []
for index in range(len(index_neighbor)):
    co_neigh.append(data[data['id'] == index_neighbor[index]]['history'])

data['history'] = data['history'].apply(lambda y :' '.join(upper_sep(eval(y))))
data['current'] = data['current'].apply(lambda y :' '.join(remove_sep(eval(y))))
data['neighbor'] = co_neigh
data['neighbor'] = data['neighbor'].apply(lambda y :' '.join(upper_sep(eval(y))))
np.save('./data/bert/15_10/test.npy',data.values.tolist())

# 7.数据处理

# 8.生成最终的数据

In [ ]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
from tqdm import tqdm
import numpy as np
import pandas as pd
train = np.load('./data/bert/15_10/train_8w.npy')
dev = np.load('./data/bert/15_10/dev_2w.npy')
# test = np.load('./data/bert/15_10_con/test.npy')
    
his_len = 150
neigh_len = 150
def pad(text,his_len):
    
    arr = text.split(' ')
    if(len(arr) < his_len):
        arr.extend(['[PAD]'] * (his_len - len(arr)))
    else:
        arr = arr[:his_len]
    return ' '.join(arr)
train_file =[]
for row in tqdm(train):
    row[0] = pad(row[0],his_len)
    row[3] =  pad(row[3],neigh_len)
    train_file.append(['[CLS] '+ row[0]+ row[3] +  ' [SEP] ' + row[1] + ' [SEP]',row[2]])
    
pd.DataFrame(train_file).to_csv('./data/bert/15_10_con/train_pair_10w.csv',index = False)
dev_file = []
for row in dev:
    row[0] = pad(row[0],his_len)
    row[3] =  pad(row[3],neigh_len)
    dev_file.append(['[CLS] '+ row[0]+ row[3] +' [SEP] ' + row[1] + ' [SEP]',row[2]])
    
pd.DataFrame(dev_file).to_csv('./data/bert/15_10_con/dev_pair_16w.csv',index = False)
test_file = []
for row in test:
    row[0] = pad(row[0],his_len)
    row[3] =  pad(row[3],neigh_len)
    test_file.append(['[CLS] '+ row[0]+ row[3] +' [SEP] ' + row[1] + ' [SEP]',row[2]])
    
pd.DataFrame(test_file).to_csv('./data/bert/15_10_con/test_pair_16w.csv',index = False)

In [ ]:
len(train_file)

In [ ]:
from keras_bert import extract_embeddings

train = np.load('./data/bert/15_10_con/train.npy')
dev = np.load('./data/bert/15_10_con/dev.npy')
test = np.load('./data/bert/15_10_con/test.npy')




In [ ]:
import pandas as pd
model_path = './data/outputs/bert_15_10_con'

from keras_bert import extract_embeddings, POOL_NSP, POOL_MAX
train_pair = pd.read_csv('./data/bert/15_10_con/train_pair.csv')
train_pair = train_pair['0'].apply(lambda x: eval(x)).values.tolist()
# dev_pair = pd.read_csv('./data/bert/15_10_con/dev_pair.csv')
# dev_pair = dev_pair['0'].apply(lambda x: eval(x)).values.tolist()
# test_pair = pd.read_csv('./data/bert/15_10_con/test_pair.csv')
# test_pair = test_pair['0'].apply(lambda x: eval(x)).values.tolist()
train_embed = []
for index,row in enumerate(train_pair):
    embed = extract_embeddings(model_path, row)[0][0]
    train_embed.append(embed)
    print(index)
    
# dev_embed = extract_embeddings(model_path, dev_pair)
# test_embed = extract_embeddings(model_path, test_pair)

In [ ]:
maxlen_history = 120
maxlen_curr = 30
import numpy as np
examples = []
step = 0
for index,row in all_train.iterrows():
    temp = []
    text = ' '.join(eval(row['history']))
    temp.append(text)
    text = ' '.join(eval(row['current']))
    temp.append(text)
    temp.append(row['label'])
    examples.append(temp)
    step += 1
    if(step % 10000 == 0):print(step)


In [ ]:

print( maxlen_history)
print(maxlen_curr)
### 训练数据裁剪
k = 1
train_1 = []
index = 0
lens = len(examples)
while index <(lens):
    flag = True
    for his in range(k):
        if examples[index+his][2] != 0:
            flag = False
            break
    if(flag):
        for pos in range(k):
            history = examples[pos + index]
            history[0] = history[0]
            train_1.append(history)
        index += k
        if(index + k >= lens):break
        while(examples[index][2] != 1):
            index += 1
            if(index + k >= lens):break
        if(index + k >= len(examples)):break
        history = examples[index]
        train_1.append(history)
    else:
        index += 1
    if(len(train_1) % 2000 == 0):print(len(train_1))
    if(index + k >= lens):break 

In [ ]:

texts = [('all work and no play', 'makes jack a dull boy'),
    ('makes jack a dull boy', 'all work and no play')]
embeddings = extract_embeddings(model_path, texts)[0][0]

In [ ]:
len(embeddings)

In [ ]:
embeddings[1][7]

In [ ]:
import torch
from transformers import BertTokenizer

model_name = './data/outputs/bert_15_10_con'  # 指定需下载的预训练模型参数

# BERT 在预训练中引入了 [CLS] 和 [SEP] 标记句子的开头和结尾
samples = ['[CLS] are you ok？ 0 [SEP]I amgo fine ! 0 [SEP]']  # 准备输入模型的语句

tokenizer = BertTokenizer.from_pretrained(model_name)

tokenized_text = [tokenizer.tokenize(i) for i in samples]

input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]
input_ids = torch.LongTensor(input_ids)
print(input_ids)

In [2]:
import pandas as pd
data = pd.read_csv('./data/train/seprate_10/all_10.csv')

In [3]:
data

,history,current,label
0,"['democrats', 'who', 'voted', 'to', 'impeach',...","['baby', 'trump', 'balloon', 'slashed', 'at', ...",0
1,"['comics', '-', 'pluggers', 'by', 'gary', 'bro...","['trump', 'says', 'china', 'trade', 'talks', '...",0
2,"['walmart', 'sparks', 'panic', 'and', 'confusi...","['highlights', 'of', 'the', '2019', 'sema', 's...",0
3,"['majority', 'disapprove', 'of', 'trump', 'sup...","['snow', 'crab', 'sells', 'for', 'record', 'pr...",0
4,"['extremely', 'disturbing', 'top', 'dems', 'al...","['kendall', 'jenner', 'wore', 'the', 'tiniest'...",0
...,...,...,...
999989,"['roger', 'stone', 'lied', 'to', 'congress', '...","['feud', 'between', 'trump', 'advisers', 'unde...",0
999990,"['back', 'to', 'blonde', 'britney', 'spears', ...","['hailey', 'bieber', 'sends', 'aunt', 'hilaria...",0
999991,"['black', 'high', 'school', 'security', 'guard...","['2019', 'peoples', 'choice', 'awards', 'fashi...",0
999992,"['the', 'woman', 'who', 'flipped', 'off', 'tru...","['my', 'sister', 'and', 'i', 'were', 'adopted'...",0
